In [ ]:
import openai
import math
import re

client = openai.OpenAI(
    api_key="sk-JBfp0POD11gasNLNn1uTiQ",  
    base_url="https://llmapi.paratera.com/v1/"  # 建议带上 https://
)

# === 工具函数 ===
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def parse_probabilities(llm_output: str):
    """
    从 LLM 文本输出中提取概率。
    比如：
    P(yes) = 0.0  
    P(no) = 1.0
    """
    matches = re.findall(r"([0-9]*\.?[0-9]+)", llm_output)
    if len(matches) >= 2:
        p1, p2 = float(matches[0]), float(matches[1])
        total = p1 + p2
        if total == 0:
            return 0.5, 0.5
        return p1 / total, p2 / total
    return 0.5, 0.5  # fallback

def parse_logits(llm_output: str):
    matches = re.findall(r"([-]?[0-9]*\.?[0-9]+)", llm_output)
    if len(matches) >= 2:
        l1, l2 = float(matches[0]), float(matches[1])
        if not math.isclose(l1 + l2, 0):
            avg = (l1 - l2) / 2
            l1, l2 = avg, -avg
        return l1, l2
    return 0.0, 0.0

def llm_judge_openai(prompt, x1, x2, method='frequency', n_sample=10):
    """
    使用 client.chat.completions.create
    method: 'frequency' | 'probability' | 'logit'
    返回: {"label": 1或0, "prob": 最大概率}
    """

    if method == 'frequency':
        votes = []
        for _ in range(n_sample):
            resp = client.chat.completions.create(
                model="Qwen3-Next-80B-A3B-Thinking",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=50,
                temperature=0.7,
            )
            text = resp.choices[0].message.content.strip()
            votes.append(1 if text.lower().startswith("yes") else 0)

        p_yes = sum(votes) / len(votes)
        p_no = 1 - p_yes
        # 取最大标签与其概率（平局默认选 yes=>1）
        if p_yes >= p_no:
            return {"label": 1, "prob": p_yes}
        else:
            return {"label": 0, "prob": p_no}

    elif method == 'probability':
        resp = client.chat.completions.create(
            model="Qwen3-Next-80B-A3B-Thinking",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200,
            temperature=0.7,
        )
        text = resp.choices[0].message.content.strip()
        #print("LLM 原始输出：\n", text)  # 调试用

        p_yes, p_no = parse_probabilities(text)
        if p_yes >= p_no:
            return {"label": 1, "prob": p_yes}
        else:
            return {"label": 0, "prob": p_no}

    elif method == 'logit':
        resp = client.chat.completions.create(
            model="Qwen3-Next-80B-A3B-Thinking",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200,
            temperature=0.7,
        )
        text = resp.choices[0].message.content.strip()
        logit_yes, logit_no = parse_logits(text)
        p_yes = sigmoid(logit_yes)
        p_no = sigmoid(logit_no)

        if p_yes >= p_no:
            return {"label": 1, "prob": p_yes}
        else:
            return {"label": 0, "prob": p_no}

    else:
        raise ValueError("Method should be one of 'frequency', 'probability', or 'logit'")



In [3]:

def check_backdoor(x1, x2, method='logit'):
    prompt = f"请判断 {x1} 与 {x2} 之间是否存在 back-door path。" \
             f"请直接输出是和否的概率对，并确保满足 Kolmogorov 公理。"
    return llm_judge_openai(prompt, x1, x2, method)

def check_independence_after_block(x1, x2, method='logit'):
    prompt = f"阻断 back-door path 后，{x1} 与 {x2} 是否独立？" \
             f"请直接输出是和否的概率对，并确保满足 Kolmogorov 公理。"
    return llm_judge_openai(prompt, x1, x2, method)

def check_latent_confounder_after_block(x1, x2, method='logit'):
    prompt = f"阻断 back-door path 后，{x1} 与 {x2} 是否存在潜在混杂因子？" \
             f"请直接输出是和否的概率对，并确保满足 Kolmogorov 公理。"
    return llm_judge_openai(prompt, x1, x2, method)

def check_causal_direction_after_block(x1, x2, method='logit'):
    prompt = f"阻断 back-door path 后，请判断 {x1} 是否会导致 {x2}。" \
             f"请直接输出是和否的概率对，并确保满足 Kolmogorov 公理。"
    return llm_judge_openai(prompt, x1, x2, method)

def check_independence(x1, x2, method='logit'):
    prompt = f"请判断 {x1} 与 {x2} 是否独立。" \
             f"请直接输出是和否的概率对，并确保满足 Kolmogorov 公理。"
    return llm_judge_openai(prompt, x1, x2, method)

def check_latent_confounder(x1, x2, method='logit'):
    prompt = f"请判断 {x1} 与 {x2} 之间是否存在潜在混杂因子。" \
             f"请直接输出是和否的概率对，并确保满足 Kolmogorov 公理。"
    return llm_judge_openai(prompt, x1, x2, method)

def check_causal_direction(x1, x2, method='logit'):
    prompt = f"请判断 {x1} 是否会导致 {x2}。" \
             f"请请直接输出是和否的概率对，并确保满足 Kolmogorov 公理。"
    return llm_judge_openai(prompt, x1, x2, method)



In [3]:
# Step 1: LLM 生成因果知识
def tree_query(x1, x2, method='probability'):
    """
    基于树状逻辑的因果方向查询器（不做阈值判断）。
    每一步子检查函数都返回: {"label": 1或0, "prob": float}

    输出:
        {
            'relation': 'x->y' | 'y->x' | 'x<->y' | 'independent',
            'confidence': float,   # 取决定该结论的那一步的 prob
            'log': [(step_name, {'label': int, 'prob': float}), ...]
        }
    """
    log = []

    # Step 1: 是否存在 backdoor path?
    res_backdoor = check_backdoor(x1, x2, method)
    log.append(("backdoor_path", res_backdoor))

    if res_backdoor["label"] == 1:
        # Step 2: 阻断路径后是否独立？
        res_ind = check_independence_after_block(x1, x2, method)
        log.append(("independent_after_block", res_ind))
        if res_ind["label"] == 1:
            return {"relation": "independent", "confidence": res_ind["prob"], "log": log}

        # Step 3: 是否存在潜在混杂因子？
        res_latent = check_latent_confounder_after_block(x1, x2, method)
        log.append(("latent_confounder_after_block", res_latent))
        if res_latent["label"] == 1:
            return {"relation": "x<->y", "confidence": res_latent["prob"], "log": log}

        # Step 4: 判断方向 (x→y?)
        res_dir = check_causal_direction_after_block(x1, x2, method)
        log.append(("x->y_after_block", res_dir))
        if res_dir["label"] == 1:
            return {"relation": "x->y", "confidence": res_dir["prob"], "log": log}
        else:
            return {"relation": "y->x", "confidence": res_dir["prob"], "log": log}

    else:
        # 不存在 backdoor path
        res_ind = check_independence(x1, x2, method)
        log.append(("independent_no_backdoor", res_ind))
        if res_ind["label"] == 1:
            return {"relation": "independent", "confidence": res_ind["prob"], "log": log}

        res_latent = check_latent_confounder(x1, x2, method)
        log.append(("latent_confounder_no_backdoor", res_latent))
        if res_latent["label"] == 1:
            return {"relation": "x<->y", "confidence": res_latent["prob"], "log": log}

        res_dir = check_causal_direction(x1, x2, method)
        log.append(("x->y_no_backdoor", res_dir))
        if res_dir["label"] == 1:
            return {"relation": "x->y", "confidence": res_dir["prob"], "log": log}
        else:
            return {"relation": "y->x", "confidence": res_dir["prob"], "log": log}


In [ ]:
import openai
import math
import re
from typing import List, Dict, Any, Tuple

client = openai.OpenAI(
    api_key="sk-JBfp0POD11gasNLNn1uTiQ",  
    base_url="https://llmapi.paratera.com/v1/"
)

# === 工具函数 ===
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def parse_probabilities(llm_output: str):
    """
    从 LLM 文本输出中提取概率。
    比如：
    P(yes) = 0.0  
    P(no) = 1.0
    """
    matches = re.findall(r"([0-9]*\.?[0-9]+)", llm_output)
    if len(matches) >= 2:
        p1, p2 = float(matches[0]), float(matches[1])
        total = p1 + p2
        if total == 0:
            return 0.5, 0.5
        return p1 / total, p2 / total
    return 0.5, 0.5  # fallback

def parse_logits(llm_output: str):
    matches = re.findall(r"([-]?[0-9]*\.?[0-9]+)", llm_output)
    if len(matches) >= 2:
        l1, l2 = float(matches[0]), float(matches[1])
        if not math.isclose(l1 + l2, 0):
            avg = (l1 - l2) / 2
            l1, l2 = avg, -avg
        return l1, l2
    return 0.0, 0.0

def llm_judge_openai(prompt, x1, x2, method='frequency', n_sample=10):
    """
    使用 client.chat.completions.create
    method: 'frequency' | 'probability' | 'logit'
    返回: {"label": 1或0, "prob": 最大概率}
    """

    if method == 'frequency':
        votes = []
        for _ in range(n_sample):
            resp = client.chat.completions.create(
                model="Qwen3-Next-80B-A3B-Thinking",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=50,
                temperature=0.7,
            )
            text = resp.choices[0].message.content.strip()
            votes.append(1 if text.lower().startswith("yes") else 0)

        p_yes = sum(votes) / len(votes)
        p_no = 1 - p_yes
        # 取最大标签与其概率（平局默认选 yes=>1）
        if p_yes >= p_no:
            return {"label": 1, "prob": p_yes}
        else:
            return {"label": 0, "prob": p_no}

    elif method == 'probability':
        resp = client.chat.completions.create(
            model="Qwen3-Next-80B-A3B-Thinking",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200,
            temperature=0.7,
        )
        text = resp.choices[0].message.content.strip()
        #print("LLM 原始输出：\n", text)  # 调试用

        p_yes, p_no = parse_probabilities(text)
        if p_yes >= p_no:
            return {"label": 1, "prob": p_yes}
        else:
            return {"label": 0, "prob": p_no}

    elif method == 'logit':
        resp = client.chat.completions.create(
            model="Qwen3-Next-80B-A3B-Thinking",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200,
            temperature=0.7,
        )
        text = resp.choices[0].message.content.strip()
        logit_yes, logit_no = parse_logits(text)
        p_yes = sigmoid(logit_yes)
        p_no = sigmoid(logit_no)

        if p_yes >= p_no:
            return {"label": 1, "prob": p_yes}
        else:
            return {"label": 0, "prob": p_no}

    else:
        raise ValueError("Method should be one of 'frequency', 'probability', or 'logit'")

# === MoE 专家定义 ===
CAUSAL_EXPERTS = {
    "graph_theory": {
        "name": "因果图论专家",
        "description": "专门分析因果图结构，精通d-分离、路径阻断、后门准则等图论概念。推理方式：系统检查所有可能的路径，分析路径上的变量类型和连接方式，使用严谨的图论推理链条。",
        "specialty": "路径分析、环路检测、d-分离判断",
        "reasoning_style": "结构化图遍历",
        "output_format": "基于图结构的二值判断"
    },
    "statistical": {
        "name": "计量统计专家", 
        "description": "专注于统计检验和概率独立性分析，擅长相关性分析、条件独立性检验、混淆变量检测。推理方式：考虑样本分布、统计显著性、置信区间等统计概念。",
        "specialty": "独立性检验、相关性分析、混淆检测",
        "reasoning_style": "概率统计推理",
        "output_format": "基于统计证据的概率判断"
    },
    "domain_knowledge": {
        "name": "领域先验专家",
        "description": "基于现实世界知识和科学常识进行因果推理，考虑时间顺序、物理可能性、生物学机制等约束。推理方式：结合文献证据、科学理论和常识性约束。",
        "specialty": "机制分析、时序推理、现实约束",
        "reasoning_style": "基于证据的归纳推理", 
        "output_format": "基于领域知识的合理性判断"
    },
    "counterfactual": {
        "name": "反事实干预专家",
        "description": "从干预和潜在结果角度分析因果关系，考虑do-calculus、随机化实验理想情况。推理方式：构建反事实场景，分析干预后的可能变化。",
        "specialty": "干预分析、潜在结果、do算子",
        "reasoning_style": "反事实思维实验",
        "output_format": "基于干预推理的因果判断"
    },
    "temporal_dynamics": {
        "name": "时间动态专家",
        "description": "专门分析时间顺序和动态过程，强调原因必须发生在结果之前，考虑延迟效应和动态反馈。推理方式：严格检查时间顺序，分析因果链的时间特性。",
        "specialty": "时序分析、动态过程、延迟效应",
        "reasoning_style": "时间序列推理", 
        "output_format": "基于时间顺序的因果判断"
    },
    "mechanism_modeling": {
        "name": "机制建模专家", 
        "description": "专注于因果机制的可解释性建模，分析中间变量、中介效应和机制路径。推理方式：构建机制框图，分析变量间的功能关系。",
        "specialty": "中介分析、机制路径、功能关系",
        "reasoning_style": "机制分解建模",
        "output_format": "基于机制完整性的判断"
    },
    "robustness_analysis": {
        "name": "稳健性检验专家",
        "description": "专门评估因果关系的稳健性和敏感性，考虑不同假设下的结果稳定性。推理方式：进行敏感性分析，检验边界条件和假设变化的影响。",
        "specialty": "敏感性分析、稳健检验、边界情况",
        "reasoning_style": "多情景验证",
        "output_format": "基于稳健性评估的判断"
    }
}

# === Router 函数（增强版）===
def expert_router(question_type: str, x1: str, x2: str) -> List[str]:
    """
    根据问题类型和变量特征选择最相关的专家
    返回专家名称列表，按相关性排序
    """
    # 基础路由规则
    routing_rules = {
        "backdoor_path": [
            "graph_theory", "statistical", "counterfactual", "temporal_dynamics", 
            "mechanism_modeling", "robustness_analysis", "domain_knowledge"
        ],
        "independence": [
            "statistical", "graph_theory", "counterfactual", "robustness_analysis",
            "temporal_dynamics", "mechanism_modeling", "domain_knowledge"
        ],
        "latent_confounder": [
            "domain_knowledge", "statistical", "mechanism_modeling", "counterfactual",
            "robustness_analysis", "graph_theory", "temporal_dynamics"
        ],
        "causal_direction": [
            "temporal_dynamics", "domain_knowledge", "counterfactual", "mechanism_modeling",
            "statistical", "graph_theory", "robustness_analysis"
        ]
    }
    
    # 获取基础专家列表
    base_experts = routing_rules.get(question_type, list(CAUSAL_EXPERTS.keys()))
    
    # 使用门诊LLM agent来智能选择专家
    try:
        clinic_recommendation = clinic_agent_recommend(question_type, x1, x2, base_experts)
        return clinic_recommendation
    except Exception as e:
        print(f"门诊agent路由失败: {e}，使用基础路由")
        # 失败时返回基础专家列表的前3个
        return base_experts[:3]

def clinic_agent_recommend(question_type: str, x1: str, x2: str, base_experts: List[str]) -> List[str]:
    """
    门诊LLM agent：根据具体变量和问题类型推荐最相关的专家
    """
    # 构建专家选择提示
    experts_description = "\n".join([
        f"- {expert}: {CAUSAL_EXPERTS[expert]['description']}" 
        for expert in base_experts
    ])
    
    clinic_prompt = f"""
作为因果推断门诊专家，你需要为以下因果分析任务选择最合适的专家组合：

**分析任务**: {question_type}
**变量对**: {x1} 和 {x2}

**可用专家列表**:
{experts_description}

**选择要求**:
1. 根据变量内容和问题类型，选择最相关的3个专家
2. 按相关性从高到低排序
3. 确保专家视角的多样性（不要选择推理方式相似的专家）
4. 考虑变量的领域特性（医学、经济、社会等）

请按照以下格式输出：
最终推荐专家: 专家1, 专家2, 专家3

**注意**: 只输出专家名称，用逗号分隔，不要添加其他文字。
"""
    
    # 调用LLM获取推荐
    resp = client.chat.completions.create(
        model="Qwen3-Next-80B-A3B-Thinking",
        messages=[{"role": "user", "content": clinic_prompt}],
        max_tokens=200,  # 减少token使用
        temperature=0.1,  # 降低随机性
    )
    
    response_text = resp.choices[0].message.content.strip()
    print(f"门诊agent原始响应: {response_text}")
    
    # 解析返回的专家列表
    recommended_experts = parse_clinic_recommendation(response_text, base_experts)
    
    print(f"门诊agent推荐: {recommended_experts}")
    
    return recommended_experts

def parse_clinic_recommendation(response_text: str, base_experts: List[str]) -> List[str]:
    """
    解析门诊agent的推荐结果 - 改进版本
    """
    # 方法1: 查找"最终推荐专家"后的内容
    if "最终推荐专家" in response_text:
        parts = response_text.split("最终推荐专家")
        if len(parts) > 1:
            expert_line = parts[1].strip().lstrip(":").strip()
            return extract_experts_from_line(expert_line, base_experts)
    
    # 方法2: 查找最后一行
    lines = [line.strip() for line in response_text.split('\n') if line.strip()]
    if lines:
        last_line = lines[-1]
        experts = extract_experts_from_line(last_line, base_experts)
        if len(experts) >= 2:
            return experts
    
    # 方法3: 在整个文本中搜索专家名称
    found_experts = []
    for expert in base_experts:
        if expert in response_text:
            found_experts.append(expert)
    
    if len(found_experts) >= 2:
        return found_experts[:3]  # 取前3个找到的专家
    
    # 如果所有方法都失败，返回基础专家前3个
    print(f"门诊agent解析不充分，使用基础专家: {base_experts[:3]}")
    return base_experts[:3]

# === 增强解析函数 ===
def extract_experts_from_line(line: str, base_experts: List[str]) -> List[str]:
    """从一行文本中提取专家名称 - 增强版本"""
    experts = []
    
    # 清理行内容，移除可能的中文标点和空格
    clean_line = line.replace('：', ':').replace('，', ',').replace(' ', '')
    
    # 多种分割方式尝试
    separators = [',', '、', ';', '，']
    
    for sep in separators:
        if sep in clean_line:
            parts = [part.strip() for part in clean_line.split(sep)]
            break
    else:
        # 如果没有分隔符，尝试按长度分割
        parts = [clean_line]
    
    for part in parts:
        # 移除可能的前缀和后缀
        clean_part = part.lower().replace('专家', '').replace('expert', '').strip()
        
        # 直接匹配专家名称
        for expert in base_experts:
            # 多种匹配方式
            if (expert in clean_part or 
                expert.replace('_', ' ') in clean_part or
                CAUSAL_EXPERTS[expert]['name'] in part):
                if expert not in experts:  # 避免重复
                    experts.append(expert)
                    break
        
        # 如果已经找到3个专家，停止搜索
        if len(experts) >= 3:
            break
    
    return experts

def create_expert_prompt(base_prompt: str, expert_type: str, x1: str, x2: str) -> str:
    """
    为不同专家创建专业化的prompt
    """
    expert_info = CAUSAL_EXPERTS[expert_type]
    
    expert_specific_prompts = {
        "graph_theory": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于图结构的二值判断

分析步骤：
1. 构建因果图模型，识别所有可能的路径
2. 应用d-分离准则分析路径阻塞情况  
3. 检查后门路径、前门路径和混杂路径
4. 基于图结构做出明确的二值判断

请严格按照图论原理进行分析，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "statistical": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于统计证据的二值判断

分析步骤：
1. 评估变量间的统计相关性
2. 考虑条件独立性和混淆因素
3. 分析统计显著性和置信度
4. 基于概率证据做出明确的二值判断

请基于统计原理进行严谨分析，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "domain_knowledge": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于领域知识的二值判断

分析步骤：
1. 调用相关领域的科学知识和常识
2. 考虑物理/生物/社会机制的合理性
3. 评估时间顺序和现实约束条件
4. 基于先验知识做出明确的二值判断

请结合现实世界知识进行推理，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "counterfactual": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于干预推理的二值判断

分析步骤：
1. 构建干预场景（do-操作）
2. 比较实际结果与反事实结果
3. 分析潜在结果分布
4. 基于干预效应做出明确的二值判断

请使用反事实推理进行分析，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "temporal_dynamics": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于时间顺序的二值判断

分析步骤：
1. 严格检查原因和结果的时间顺序
2. 分析延迟效应和动态过程
3. 考虑时间序列的因果结构
4. 基于时间约束做出明确的二值判断

请重点分析时间维度，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "mechanism_modeling": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于机制完整性的二值判断

分析步骤：
1. 识别可能的中间机制和中介变量
2. 分析因果链的功能完整性
3. 评估机制路径的合理性
4. 基于机制可解释性做出明确的二值判断

请专注于机制分析，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "robustness_analysis": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于稳健性评估的二值判断

分析步骤：
1. 测试不同假设条件下的结果稳定性
2. 进行敏感性分析和边界检验
3. 评估结论的稳健程度
4. 基于稳健性评估做出明确的二值判断

请重点分析结论的可靠性，直接输出是或否（Yes/No）。\n\n{base_prompt}"""
    }
    
    return expert_specific_prompts.get(expert_type, base_prompt)

# === MoE 集成函数 ===
def aggregate_expert_judgments(expert_results: List[Dict]) -> Dict:
    """
    整合多个专家的判断结果
    """
    if not expert_results:
        return {"label": 0, "prob": 0.5}
    
    # 简单加权平均
    total_prob_yes = 0
    total_weight = 0
    
    for result in expert_results:
        weight = result.get("confidence", 1.0)
        prob_yes = result["prob"] if result["label"] == 1 else 1 - result["prob"]
        total_prob_yes += prob_yes * weight
        total_weight += weight
    
    aggregated_prob_yes = total_prob_yes / total_weight if total_weight > 0 else 0.5
    
    if aggregated_prob_yes >= 0.5:
        return {"label": 1, "prob": aggregated_prob_yes}
    else:
        return {"label": 0, "prob": 1 - aggregated_prob_yes}

# === 修改后的 run_step_with_moe 函数 ===
def run_step_with_moe(base_prompt: str, x1: str, x2: str, question_type: str, method: str = 'frequency') -> Dict:
    """
    使用MoE架构运行单个判断步骤
    """
    # 注意：这里我们不需要修改，因为base_prompt已经包含了all_variables
    # 1. 路由选择专家
    selected_experts = expert_router(question_type, x1, x2)
    print(f"为问题 '{question_type}' 选择的专家: {selected_experts}")
    
    # 2. 执行专家判断
    expert_results = []
    for expert in selected_experts:  # 使用所有推荐的专家
        expert_prompt = create_expert_prompt(base_prompt, expert, x1, x2)
        try:
            result = llm_judge_openai(expert_prompt, x1, x2, method)
            result["expert"] = expert
            result["confidence"] = 1.0
            expert_results.append(result)
            print(f"专家 {expert} 判断完成: label={result['label']}, prob={result['prob']}")
        except Exception as e:
            print(f"专家 {expert} 执行失败: {e}")
            continue
    
    # 3. 如果没有专家成功，使用默认方法
    if not expert_results:
        print("所有专家执行失败，使用默认方法")
        return llm_judge_openai(base_prompt, x1, x2, method)
    
    # 4. 整合专家意见
    final_result = aggregate_expert_judgments(expert_results)
    final_result["expert_results"] = expert_results
    
    print(f"专家整合结果: label={final_result['label']}, prob={final_result['prob']}")
    return final_result

# === 修改后的具体判断函数 ===
def check_backdoor(x1, x2, all_variables, method='frequency'):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

请判断在这些变量中，变量 {x1} 和 {x2} 之间是否存在 back-door path（后门路径）。

后门路径是指从 {x1} 到 {x2} 的路径，其中包含指向 {x1} 的箭头，且这条路径没有被阻断。

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "backdoor_path", method)

def check_independence_after_block(x1, x2, all_variables, method='frequency'):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

如果阻断了 {x1} 和 {x2} 之间的所有 back-door path，那么 {x1} 与 {x2} 是否条件独立？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "independence", method)

def check_latent_confounder_after_block(x1, x2, all_variables, method='frequency'):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

阻断了 {x1} 和 {x2} 之间的所有 back-door path 后，是否仍然存在未观察到的潜在混杂因子同时影响 {x1} 和 {x2}？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "latent_confounder", method)

def check_causal_direction_after_block(x1, x2, all_variables, method='frequency'):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

阻断了 {x1} 和 {x2} 之间的所有 back-door path 后，请判断 {x1} 是否因果导致 {x2}？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "causal_direction", method)

def check_independence(x1, x2, all_variables, method='frequency'):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

请判断变量 {x1} 和 {x2} 是否独立？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "independence", method)

def check_latent_confounder(x1, x2, all_variables, method='frequency'):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

请判断变量 {x1} 和 {x2} 之间是否存在未观察到的潜在混杂因子？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "latent_confounder", method)

def check_causal_direction(x1, x2, all_variables, method='frequency'):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

请判断 {x1} 是否因果导致 {x2}？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "causal_direction", method)

# === 修改后的 tree_query 函数 ===
def tree_query(x1, x2, all_variables, method='frequency'):
    """
    基于树状逻辑的因果方向查询器（使用MoE架构）
    """
    log = []

    # Step 1: 是否存在 backdoor path?
    print("=== Step 1: 检查后门路径 ===")
    res_backdoor = check_backdoor(x1, x2, all_variables, method)
    log.append(("backdoor_path", res_backdoor))

    if res_backdoor["label"] == 1:
        print("存在后门路径，进入阻断后分析路径")
        # Step 2: 阻断路径后是否独立？
        print("=== Step 2: 阻断后检查独立性 ===")
        res_ind = check_independence_after_block(x1, x2, all_variables, method)
        log.append(("independent_after_block", res_ind))
        if res_ind["label"] == 1:
            return {"relation": "independent", "confidence": res_ind["prob"], "log": log}

        # Step 3: 是否存在潜在混杂因子？
        print("=== Step 3: 检查潜在混杂因子 ===")
        res_latent = check_latent_confounder_after_block(x1, x2, all_variables, method)
        log.append(("latent_confounder_after_block", res_latent))
        if res_latent["label"] == 1:
            return {"relation": "x<->y", "confidence": res_latent["prob"], "log": log}

        # Step 4: 判断方向 (x→y?)
        print("=== Step 4: 判断因果方向 ===")
        res_dir = check_causal_direction_after_block(x1, x2, all_variables, method)
        log.append(("x->y_after_block", res_dir))
        if res_dir["label"] == 1:
            return {"relation": "x->y", "confidence": res_dir["prob"], "log": log}
        else:
            return {"relation": "y->x", "confidence": res_dir["prob"], "log": log}

    else:
        print("不存在后门路径，进入直接分析路径")
        # 不存在 backdoor path
        res_ind = check_independence(x1, x2, all_variables, method)
        log.append(("independent_no_backdoor", res_ind))
        if res_ind["label"] == 1:
            return {"relation": "independent", "confidence": res_ind["prob"], "log": log}

        res_latent = check_latent_confounder(x1, x2, all_variables, method)
        log.append(("latent_confounder_no_backdoor", res_latent))
        if res_latent["label"] == 1:
            return {"relation": "x<->y", "confidence": res_latent["prob"], "log": log}

        res_dir = check_causal_direction(x1, x2, all_variables, method)
        log.append(("x->y_no_backdoor", res_dir))
        if res_dir["label"] == 1:
            return {"relation": "x->y", "confidence": res_dir["prob"], "log": log}
        else:
            return {"relation": "y->x", "confidence": res_dir["prob"], "log": log}
# === 使用示例 ===
if __name__ == "__main__":
    # 定义完整的变量集合
    all_variables = ["冰淇淋销量", "溺水人数", "温度"]
    
    # 测试MoE框架
    print("开始因果推断分析...")
    result = tree_query("冰淇淋销量", "溺水人数", all_variables, method='frequency')
    print("\n=== 最终结果 ===")
    print("关系:", result["relation"])
    print("置信度:", result["confidence"])
    print("\n=== 详细执行日志 ===")
    for step_name, step_result in result["log"]:
        print(f"{step_name}: {step_result}")
        if "expert_results" in step_result:
            print("  专家详情:", [(r["expert"], r["label"], r["prob"]) for r in step_result["expert_results"]])

开始因果推断分析...
=== Step 1: 检查后门路径 ===
门诊agent原始响应: graph_theory,domain_knowledge,statistical
门诊agent推荐: ['graph_theory', 'domain_knowledge', 'statistical']
为问题 'backdoor_path' 选择的专家: ['graph_theory', 'domain_knowledge', 'statistical']
专家 graph_theory 判断完成: label=1, prob=1.0
专家 domain_knowledge 判断完成: label=1, prob=1.0
专家 statistical 判断完成: label=1, prob=1.0
专家整合结果: label=1, prob=1.0
存在后门路径，进入阻断后分析路径
=== Step 2: 阻断后检查独立性 ===
门诊agent原始响应: domain_knowledge, graph_theory, statistical
门诊agent推荐: ['domain_knowledge', 'graph_theory', 'statistical']
为问题 'independence' 选择的专家: ['domain_knowledge', 'graph_theory', 'statistical']
专家 domain_knowledge 判断完成: label=1, prob=1.0
专家 graph_theory 判断完成: label=1, prob=1.0
专家 statistical 判断完成: label=1, prob=1.0
专家整合结果: label=1, prob=1.0

=== 最终结果 ===
关系: independent
置信度: 1.0

=== 详细执行日志 ===
backdoor_path: {'label': 1, 'prob': 1.0, 'expert_results': [{'label': 1, 'prob': 1.0, 'expert': 'graph_theory', 'confidence': 1.0}, {'label': 1, 'prob': 1.0, 'expert': 'domain

In [4]:
from itertools import combinations

def compute_all_causal_relations(variables, method='probability'):
    """
    计算图中每两个变量之间的因果关系，使用tree_query函数。
    
    输出:
        {
            (x1, x2): {
                'relation': 'x->y' | 'y->x' | 'x<->y' | 'independent',
                'confidence': float,
                'log': [(step_name, {'label': int, 'prob': float}), ...]
            },
            ...
        }
    """
    all_relations = {}

    # 生成所有变量的组合 C(n, 2)
    for x1, x2 in combinations(variables, 2):
        # 进行 tree_query
        result = tree_query(x1, x2, method)
        
        # 存储结果
        all_relations[(x1, x2)] = result

    return all_relations


In [5]:
variables = ['气温', '冰淇淋销量', '溺水人数']
relations = compute_all_causal_relations(variables)

for (x1, x2), relation in relations.items():
    print(f"Relation between {x1} and {x2}: {relation['relation']} (Confidence: {relation['confidence']})")


LLM 原始输出：
 0.85, 0.15
LLM 原始输出：
 0,1
LLM 原始输出：
 0.0 1.0
LLM 原始输出：
 1.0, 0.0
LLM 原始输出：
 (1, 0)
LLM 原始输出：
 0.0,1.0
LLM 原始输出：
 0.5, 0.5
LLM 原始输出：
 1.0, 0.0
LLM 原始输出：
 1,0
Relation between 气温 and 冰淇淋销量: x->y (Confidence: 1.0)
Relation between 气温 and 溺水人数: x<->y (Confidence: 0.5)
Relation between 冰淇淋销量 and 溺水人数: independent (Confidence: 1.0)


In [9]:
tree_query('氟化物','蛀牙', method='probability')

LLM 原始输出：
 1.386, -1.386
LLM 原始输出：
 在因果推断中，阻断 back-door path（后门路径）通常是通过条件在混杂变量（confounder）上来实现，以消除混杂偏差，从而准确估计氟化物（F）对蛀牙（C）的因果效应。在标准因果图（例如，氟化物 → 蛀牙，并有混杂变量 U，如社会经济状态（SES），其中 U → 氟化物 和 U → 蛀牙）中，阻断 back-door path（例如，通过条件在 U 上）后，氟化物与蛀牙之间仍存在直接因果路径（氟化物 → 蛀牙）。因此，氟化物与蛀牙在条件上（给定 U）并不独立，除非因果效应为零（即氟化物对蛀牙无影响）。在现实中，氟化物通常被认为能减少蛀牙风险，因此因果效应存在，独立性不成立。

### 回答：
- **是否独立？** 否，阻断 back-door path 后


{'relation': 'independent',
 'confidence': 0.5,
 'log': [('backdoor_path', {'label': 1, 'prob': 0.5}),
  ('independent_after_block', {'label': 1, 'prob': 0.5})]}

In [1]:
# Step 2: 多校准 (multi-Calibration)

# 获取语义向量
# === Word2Vec提取语义向量 ===
def get_word2vec_embeddings(vars_list, model_name="word2vec-google-news-300"):
    """
    使用预训练Word2Vec模型提取变量语义嵌入向量
    输入：vars_list - 变量列表
    输出：{变量名: 128维向量}
    """
    # 加载预训练Word2Vec模型（首次运行会自动下载，约1.6GB）
    print(f"加载预训练Word2Vec模型：{model_name}...")
    w2v_model = load(model_name)
    embeddings = []

    for var in vars_list:
        # 提取向量：若变量在词汇表中，直接获取；否则生成随机向量
        if var in w2v_model.key_to_index:
            vec = w2v_model[var]
        else:
            # 词汇表外变量：生成-1到1的随机向量（符合原需求范围）
            vec = np.random.uniform(low=-1.0, high=1.0, size=300)
            print(f"变量「{var}」不在Word2Vec词汇表中，使用随机向量替代")

        # 调整为128维（原模型为300维，截断或填充）
        vec = vec[:128] if len(vec) >= 128 else np.pad(vec, (0, 128 - len(vec)), 'constant')
        # 归一化到-1到1范围
        vec = vec / np.max(np.abs(vec)) if np.max(np.abs(vec)) != 0 else vec
        # 保留4位小数（与原需求一致）
        vec = np.round(vec, 4)
        embeddings.append(vec)
        print(f"成功生成变量「{var}」的Word2Vec语义向量（128维）")

    return {var: emb for var, emb in zip(vars_list, embeddings)}

#  用KMeans进行聚类
def clustering(causal_knowledge, var_embeddings, n_clusters=2):
    pair_list = list(causal_knowledge.keys())
    pair_features = []
    for pair in pair_list:
        x1, x2 = pair
        if x1 not in var_embeddings or x2 not in var_embeddings:
            raise ValueError(f"变量{x1}/{x2}无语义向量，请先调用get_llm_embeddings")
        feat = (var_embeddings[x1] + var_embeddings[x2]) / 2
        pair_features.append(feat)
    pair_features = np.array(pair_features)

    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(pair_features)

    cluster_to_pairs = {}
    for pair, label in zip(pair_list, cluster_labels):
        cluster_id = f"S_{label+1}"
        if cluster_id not in cluster_to_pairs:
            cluster_to_pairs[cluster_id] = []
        cluster_to_pairs[cluster_id].append(pair)

    print(f"\n=== 硬聚类结果（{n_clusters}个集合S） ===")
    for cluster_id, pairs in cluster_to_pairs.items():
        print(f"集合{cluster_id}（{len(pairs)}个变量对）：{pairs}")
    return cluster_to_pairs, {pair: f"S_{label+1}" for pair, label in zip(pair_list, cluster_labels)}

class ClusterInfo:
    """封装聚类相关信息，用于alpha_calibrate函数调用"""
    def __init__(self, cluster_to_pairs, pair_to_cluster, var_embeddings, N, alpha, perturbation_strength):
        self.cluster_to_pairs = cluster_to_pairs  # {簇ID: [变量对列表]}
        self.pair_to_cluster = pair_to_cluster  # {变量对: 簇ID}
        self.var_embeddings = var_embeddings    # 变量语义向量（用于聚类验证）
        self.N = N                              # 总变量对数量（universe大小）
        self.alpha = alpha                      # 算法3.1的α参数
        self.perturbation_strength = perturbation_strength  # 首次迭代扰动强度，仅用于此代码运行

#这里应该是调用统计查询获得真实基准p*(S)的函数，这里仅供代码运行
def statistical_query_oracle(
    S_pairs: list,
    prob_type: str,
    causal_knowledge: dict,
    tau: float,
    N: int,
    iteration: int,
    perturbation_strength: float = 0.03
) -> float:
    """算法3.1的统计查询接口：返回真实基准p*_S的近似值（含首次扰动）"""
    # 基础值：原始概率的簇内均值
    base_p_star_S = np.mean([causal_knowledge[pair][prob_type] for pair in S_pairs])
    S_size = len(S_pairs)

    # 首次迭代添加语义合理的扰动
    if iteration == 1:
        if prob_type == "causal":
            perturbation = perturbation_strength if base_p_star_S > 0.5 else -perturbation_strength
        elif prob_type == "independent":
            perturbation = perturbation_strength if base_p_star_S > 0.5 else -perturbation_strength
        else:
            perturbation = np.random.choice([-perturbation_strength, perturbation_strength])
        perturbed_p_star_S = np.clip(base_p_star_S + perturbation, 0.01, 0.99)
        p_star_S = perturbed_p_star_S
    else:
        p_star_S = base_p_star_S

    # 误差控制（符合算法3.1的统计查询容忍度）
    max_error = tau * N / S_size
    error = np.random.uniform(-max_error, max_error)
    p_star_S_with_error = np.clip(p_star_S + error, 0.01, 0.99)

    return p_star_S_with_error


def multi_calibration(
    causal_knowledge: dict,
    n_clusters: int = 2,
    alpha: float = 0.05,
    max_iter: int = 1000,
    perturbation_strength: float = 0.03
) -> dict:
    """
    使用校准函数调整推导的因果概率，确保与真实的因果关系匹配
    校准过程：
    - 使用嵌入的语义向量对因果知识进行硬聚类；
    - 迭代调用alpha_calibrate，对每种概率类型分簇校准；
    - 归一化概率，确保三种概率之和为1。
    
    输入：
    - causal_knowledge: LLM生成的因果知识字典，格式：
      {
          ('X1', 'X2'): {
              'independent': p_independent,
              'latent': p_latent,
              'causal': p_causal
          },
          ...
      }
    - n_clusters: 硬聚类的簇数（默认2）
    - alpha: 校准精度参数（默认0.05）
    - max_iter: 最大迭代次数（默认1000）
    - perturbation_strength: 首次迭代的扰动强度（默认0.03）
    
    输出：
    - calibrated_probabilities: 校准后的因果概率字典（格式与输入一致）
    """
    # 生成所有变量的语义向量（聚类依赖）
    print("=== 生成变量语义向量 ===")
    all_vars = list(set([var for pair in causal_knowledge.keys() for var in pair]))
    var_embeddings = get_word2vec_embeddings(all_vars)

    # 对变量对进行硬聚类（划分簇S）
    print("\n=== 步骤2：对变量对进行硬聚类 ===")
    cluster_to_pairs, pair_to_cluster = clustering(
        causal_knowledge=causal_knowledge,
        var_embeddings=var_embeddings,
        n_clusters=n_clusters
    )

    # 初始化参数与ClusterInfo对象（封装聚类信息）
    N = len(causal_knowledge)  # 总变量对数量
    cluster_info = ClusterInfo(
        cluster_to_pairs=cluster_to_pairs,
        pair_to_cluster=pair_to_cluster,
        var_embeddings=var_embeddings,
        N=N,
        alpha=alpha,
        perturbation_strength=perturbation_strength
    )
    calibrated_probabilities = {pair: prob.copy() for pair, prob in causal_knowledge.items()}
    updated = True
    iteration = 0

    # 迭代调用alpha_calibrate进行多校准
    print("\n=== 步骤3：迭代执行α校准 ===")
    while updated and iteration < max_iter:
        updated = False
        iteration += 1
        print(f"\n=== 迭代{iteration}/{max_iter} ===")

        # 遍历所有变量对，对每种概率类型调用alpha_calibrate
        temp_calibrated = calibrated_probabilities.copy()  # 临时存储本轮更新结果（避免迭代中相互影响）
        for pair, probabilities in calibrated_probabilities.items():
            # 对三种概率类型分别校准
            for prob_type in ['independent', 'latent', 'causal']:
                original_prob = probabilities[prob_type]
                # 调用alpha_calibrate进行校准
                calibrated_prob = alpha_calibrate(
                    probability=original_prob,
                    pair=pair,
                    prob_type=prob_type,
                    causal_knowledge=causal_knowledge,
                    cluster_info=cluster_info,
                    iteration=iteration,
                    calibrated_probs=calibrated_probabilities  # 传入当前所有校准概率，用于计算x_S
                )
                # 暂存校准结果（本轮内不覆盖，避免影响其他变量对的x_S计算）
                temp_calibrated[pair][prob_type] = calibrated_prob
                # 若有更新，标记本轮为更新状态
                if abs(calibrated_prob - original_prob) > 1e-6:  # 忽略微小浮点误差
                    updated = True

        # 更新校准概率，并对每个变量对的概率归一化（确保和为1）
        for pair in temp_calibrated:
            total = sum(temp_calibrated[pair].values())
            if total > 0:
                temp_calibrated[pair] = {
                    k: round(v / total, 3)
                    for k, v in temp_calibrated[pair].items()
                }
        calibrated_probabilities = temp_calibrated

    # 输出迭代终止信息
    print(f"\n=== 迭代终止 ===")
    print(f"总迭代次数：{iteration}次（{'已收敛' if not updated else '达到最大迭代次数'}）")
    return calibrated_probabilities

# 校准函数，模拟alpha校准效果
def alpha_calibrate(
    probability: float,
    pair: tuple,
    prob_type: str,
    causal_knowledge: dict,
    cluster_info: ClusterInfo,
    iteration: int,
    calibrated_probs: dict
) -> float:
    """
    算法3.1的α校准核心逻辑：
    1. 根据变量对所属簇，获取簇内信息；
    2. 调用统计查询获取真实基准p*_S；
    3. 计算当前簇内预测均值x_S，判断偏差是否超阈值；
    4. 超阈值则更新概率，返回校准后的值。
    """
    # 获取当前变量对的簇信息
    cluster_id = cluster_info.pair_to_cluster[pair]
    S_pairs = cluster_info.cluster_to_pairs[cluster_id]  # 簇内所有变量对
    S_size = len(S_pairs)
    N = cluster_info.N
    alpha = cluster_info.alpha
    perturbation_strength = cluster_info.perturbation_strength

    # 计算算法3.1的关键参数
    gamma = S_size / N
    tau = alpha * gamma / 4  # 统计查询容忍度
    threshold = alpha * S_size - tau * N  # 偏差阈值

    # 调用统计查询获取真实基准p*_S
    p_star_S = statistical_query_oracle(
        S_pairs=S_pairs,
        prob_type=prob_type,
        causal_knowledge=causal_knowledge,
        tau=tau,
        N=N,
        iteration=iteration,
        perturbation_strength=perturbation_strength
    )

    # 计算当前簇内预测均值x_S（基于所有变量对的当前校准概率）
    x_S = np.mean([calibrated_probs[pair_in_S][prob_type] for pair_in_S in S_pairs])

    # 计算偏差并判断是否更新
    delta_S = p_star_S - x_S
    print(f"簇{cluster_id}（{prob_type}）：变量对{pair} → x_S={x_S:.3f}，ΔS={delta_S:.3f}，阈值={threshold:.3f}")

    if abs(delta_S) > threshold:
        # 偏差超阈值，计算更新步长（均匀分配到簇内所有变量对）
        update_step = delta_S / S_size
        calibrated_prob = probability + update_step
        print(f"簇{cluster_id}（{prob_type}）：变量对{pair} → 原始={probability:.3f}，更新={update_step:.4f}")
    else:
        # 偏差未超阈值，不更新
        calibrated_prob = probability
        print(f"簇{cluster_id}（{prob_type}）：变量对{pair} → 偏差未超阈值，不更新")

    # 确保概率在[0.01, 0.99]范围内（避免极端值）
    calibrated_prob = max(0.01, min(calibrated_prob, 0.99))
    return calibrated_prob

In [ ]:
# Step 3: 形成先验因果图
def create_prior_causal_graph(calibrated_probabilities):
    """
    根据校准后的因果概率生成先验因果图
    - 使用三个概率判断每对变量之间的因果关系（独立性、潜在混杂变量、因果方向）。
    - 根据综合判断结果生成因果图，可能会有双向箭头（<->）表示不确定或相互作用。
    
    输入：
    - calibrated_probabilities: 每对变量的校准后概率字典，
        格式：{
            ('X1', 'X2'): {
                'independent': p_independent,
                'latent': p_latent,
                'causal': p_causal
            },
            ...
        }
    输出：
    - prior_graph: 先验因果图，格式为字典，键是变量对，值是因果关系（X->Y, Y->X, <->, independent）
    """
    prior_graph = {}
    
    # 遍历每对变量，判断因果关系
    for pair, probabilities in calibrated_probabilities.items():
        p_independent = probabilities['independent']  # 独立性概率
        p_latent = probabilities['latent']  # 潜在混杂变量概率
        p_causal = probabilities['causal']  # 因果方向概率
        
        # 判断因果关系
        if p_independent > 0.5:
            # 如果独立性概率大于0.5，认为X1和X2是独立的
            prior_graph[pair] = "independent"
        elif p_latent > 0.5:
            # 如果潜在混杂变量概率大于0.5，认为存在潜在混杂变量
            prior_graph[pair] = "<->"  # 双向箭头表示不确定或相互作用
        else:
            # 根据因果关系概率判断因果方向
            if p_causal > 0.5:
                # 如果因果概率大于0.5，认为X1导致X2
                prior_graph[pair] = f"{pair[0]}->{pair[1]}"
            elif p_causal < 0.5:
                # 如果因果概率小于0.5，认为X2导致X1
                prior_graph[pair] = f"{pair[1]}->{pair[0]}"
            else:
                # 如果因果概率接近0.5，认为两者之间相互作用
                prior_graph[pair] = "<->"
    
    return prior_graph


In [ ]:
# Step 4: 使用BCCD结合数据生成后验因果图
def generate_post_causal_graph(prior_graph, data):
    """
    使用BCCD（贝叶斯因果链发现）结合数据生成后验因果图
    - 结合先验因果图和实际数据，推导出后验因果关系。
    - 应用约束（如无环性约束），确保生成的因果图合理。
    
    输入：先验因果图、数据
    输出：后验因果图（推导出的因果关系图，满足约束条件）
    """
    # 将先验因果图应用于数据，使用BCCD进一步推导因果关系
    post_causal_graph = bccd_inference(data, prior_graph)
    
    # 对后验图进行约束，确保没有环路等不合理的因果关系
    post_causal_graph = apply_constraints(post_causal_graph)
    
    return post_causal_graph
